*This code creates the baseline (non-time dependent) variables for the schisto sdm analysis at a 1km resolution*

In [ ]:
# this code will create rasters that is distance to urban edges in Brazil
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

**SET UP DATA**

In [ ]:
# Add shapefiles to make it easy to reduce images to Brazil
region = ee.FeatureCollection('USDOS/LSIB/2017').filter(ee.Filter.eq('COUNTRY_NA', 'Brazil'))

# All occurrence points snail occurrence feature collection w/1km buffer
all_sdm_points = ee.FeatureCollection('users/cglidden/all_points_schisto_sdm')

# Functions to create buffer zone around each occurrence point
def buffer_points(radius, bounds):
    def inner(pt):
        pt = ee.Feature(pt)
        return bounds and pt.buffer(radius).bounds() or pt.buffer(radius)
    return inner

# Asset of regions for which you want to calculate statistics - radius of 1km^2
bufferedPoints = all_sdm_points.map(buffer_points(500, True))  # Square pixel

# Create land mask (remove ocean from data by masking)
mask = ee.Image.constant(1).clip(region).mask()

# Create a source image where the geometry is 1, everything else is 0 - for cost distance mapping, unmasking, etc.
sources = ee.Image.constant(1).toByte().paint(region.geometry(), 1)

# Mask the sources image with itself.
sources = sources.selfMask()

**TOPOLOGY DATA**

In [ ]:
DEM_data = (ee.Image('NASA/NASADEM_HGT/001')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region));

elevation = (ee.Image('NASA/NASADEM_HGT/001').select('elevation')
            .resample("bilinear")
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region).
            rename('elevation'));

slope = (ee.Terrain.slope(elevation)
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('slope'));

aspect = (ee.Terrain.aspect(elevation)
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('aspect'));

**SOIL DATA**

In [ ]:
clay = (ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").select('b0')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('clay'));

sand = (ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").select('b0').rename('sand')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('sand'));

soilWater = (ee.Image("OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01").select('b0').rename('soilWater')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('soilWater'));

soilCarbon = (ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select('b0').rename('soilCarbon')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('soilCarbon'));

pH = (ee.Image("OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02").select('b0').rename('pH')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('pH'));

bulk = (ee.Image("OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02").select('b0').rename('bulk')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('bulk'));




**HYDROLOGY / WATER FEATURES**

In [ ]:
MERIT = ee.Image('MERIT/Hydro/v1_0_1');

upa = (MERIT.select('upa')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region).
            rename('upa'));

hnd = (MERIT.select('hnd')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.max())
            .clip(region)
            .rename('hnd'));

######### River networks
sheds = ee.FeatureCollection("WWF/HydroSHEDS/v1/FreeFlowingRivers").filterMetadata('COUNTRY', 'equals', 'Brazil');

# Visually check different levels of river order to verify result is as intended
orderSheds = sheds.filterMetadata('RIV_ORD', 'not_greater_than', 8)

# Convert features to image, set river values to 1, mask the rest
shed_im = (sheds
    .map(lambda feat: feat.buffer(50))
    .reduceToImage(
        properties=['RIV_ORD'],
        reducer=ee.Reducer.first()
    )
)

# Convert features to image, set river values to 0, and all other areas to 1
ordShed_im = (orderSheds
    .reduceToImage(
        properties=['BB_ID'],
        reducer=ee.Reducer.first()
    )
    .multiply(0)
    .rename('BB_ID')
    .unmask(sources)
)

overRivers = ee.Image().toByte().paint(region.geometry(), 20000);

riverDist = (ordShed_im.cumulativeCost(
  source = ordShed_im.eq(0),
  maxDistance = 10 * 1000)
.rename('riverDist').toFloat().unmask(overRivers));

#### Water surface
waterJRC = ee.Image('JRC/GSW1_3/GlobalSurfaceWater');

occ = (waterJRC.select('occurrence').unmask(sources.multiply(0)).updateMask(mask)
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('waterOccurrence'));

seas = (waterJRC.select('seasonality').unmask(sources.multiply(0)).updateMask(mask)
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('waterSeasonality'));

rec = (waterJRC.select('recurrence').unmask(sources.multiply(0)).updateMask(mask)
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('waterRecurrence'));


**EXTRA VAR**

In [ ]:
## global human modification dataset - already at 1km
GHM = ee.ImageCollection('CSP/HM/GlobalHumanModification').first();


**COMPILE FINAL IMAGE AND EXPORT TO DRIVE**

In [ ]:
final_image = (GHM.addBands([elevation, slope, aspect, occ,
  seas, rec, riverDist, clay, sand, soilWater, soilCarbon, pH, bulk, upa, hnd]))


# Export image to drive
export_task = ee.batch.Export.table.toDrive(
    collection=final_image.reduceRegions(
        collection=bufferedPoints,
        reducer=ee.Reducer.mean(),
        scale=1000,
        tileScale=16
    ),
    description='all_points_sdm_misc_vars',
    folder='final_schisto_data',
    fileFormat='SHP'
)

# Start the export task
export_task.start()
